### Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences text or speech)

Another common term for NLP problems is sequence to sequence problems (seq2seq)

In [1]:
# Check for GPU
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1650 (UUID: GPU-890aadbc-dc9a-cfdd-ce0d-dd3bdfa0ac28)


In [17]:
!wget https://github.com/jrassoul/TensorFlow_Developer_Certificate_ZTM/blob/12a3f6e79366dd14dea8c3c61da8cb9d23ed9631/helper_functions.py

'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.


In [3]:
!pip install wget

  Using cached wget-3.2-py3-none-any.whl


In [14]:
 ## GET helper functions
import sys
sys.path.append(r"C:\Users/wjub/01workspace/TensorFlow Developer Certificate ZTM")
import winsound
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import helper_functions

In [16]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaste or not disasterrs)

See the original source here : https://www.kaggle.com/competitions/nlp-getting-started/data

In [19]:
helper_functions.download_file("https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip","nlp_getting_started.zip")

In [20]:
helper_functions.unzip_data("nlp_getting_started.zip")

## Visualizing a text data 

To visualize our text samples, we first have to read them in, one way to do so would be to use Python :

So another way to do this is to use pandas.

In [26]:
import pandas as pd
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
train_df.head()
# train_df["text"][0]

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [28]:
# Shuffle training dataframe
# frac : nb of % to shuffle 1=100%
train_df_shuffled = train_df.sample(frac=1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [29]:
# What does the test dataframe look like ?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [32]:
# How many examples of each class ?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [33]:
# How many total sample ?
len(train_df), len(test_df)

(7613, 3263)

In [40]:
#  Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of  samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target:{target}", "(real diaster)" if target > 0 else "(not real diaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target:0 (not real diaster)
Text:
no but seriously I will electrocute half of UK Army's so I can touch bangtan i do not play games when it comes to bts

---

Target:1 (real diaster)
Text:
After shooting event at the theater @Starbucks is giving FREE COFFEE to all first responders police &amp; firefighters. AntiochHickoryHollow#TN

---

Target:1 (real diaster)
Text:
News@ Refugio oil spill may have been costlier bigger than projected http://t.co/jhpdSSVhvE

---

Target:1 (real diaster)
Text:
But if you build an army of 100 dogs and their leader is a lion all dogs will fight like a lion.

---

Target:1 (real diaster)
Text:
Australia's Ashes disaster - how the collapse unfolded at Trent Bridge... http://t.co/Dq3ddGvgBF
 #cricket

---

